In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import torch
import torchvision
import torchvision.transforms as transforms
from torch.utils.data import DataLoader, random_split
from torchvision.datasets import ImageFolder
import torch.nn as nn
import torch.optim as optim
from torch.cuda.amp import GradScaler, autocast
import matplotlib.pyplot as plt
import time


In [3]:
# Data loading
def load_data(data_dir, batch_size, num_workers):
    transform = transforms.Compose([
        transforms.Resize((224, 224)),
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
    ])
    dataset = ImageFolder(root=data_dir, transform=transform)
    train_size = int(0.8 * len(dataset))
    test_size = len(dataset) - train_size
    train_dataset, test_dataset = random_split(dataset, [train_size, test_size])

    train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True, num_workers=num_workers)
    test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False, num_workers=num_workers)

    return train_loader, test_loader

In [4]:
# Model definition
class AnimalClassifier(nn.Module):
    def __init__(self):
        super(AnimalClassifier, self).__init__()
        self.model = torchvision.models.resnet50(pretrained=True)
        self.model.fc = nn.Linear(self.model.fc.in_features, 50)  # Adjusting the final layer for 50 classes

    def forward(self, x):
        return self.model(x)

In [5]:
# Model training
def train_model(model, dataloader, optimizer, criterion, device, epochs):
    model.train()
    scaler = GradScaler()
    start_time = time.time()

    for epoch in range(epochs):
        for i, (images, labels) in enumerate(dataloader):
            images, labels = images.to(device), labels.to(device)
            optimizer.zero_grad()

            with autocast():
                outputs = model(images)
                loss = criterion(outputs, labels)

            scaler.scale(loss).backward()
            scaler.step(optimizer)
            scaler.update()

            # Print the loss every 20 steps
            if (i + 1) % 20 == 0:
                print(f"Epoch [{epoch+1}/{epochs}], Step [{i+1}/{len(dataloader)}], Loss: {loss.item():.4f}")

    end_time = time.time()
    training_time = end_time - start_time
    print(f"Training completed in {training_time:.2f} seconds")


In [6]:
# Model evaluation
def evaluate_model(model, dataloader, device):
    model.eval()
    correct = 0
    total = 0
    with torch.no_grad():
        for images, labels in dataloader:
            images, labels = images.to(device), labels.to(device)
            outputs = model(images)
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
    accuracy = 100 * correct / total
    print(f'Accuracy: {accuracy}%')
    return accuracy

In [7]:
# Main execution block
def main():
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    train_loader, test_loader = load_data('/content/drive/My Drive/JPEGImages', batch_size=64, num_workers=4)
    model = AnimalClassifier().to(device)
    optimizer = optim.Adam(model.parameters(), lr=0.001)
    criterion = nn.CrossEntropyLoss()
    train_model(model, train_loader, optimizer, criterion, device, epochs=10)
    evaluate_model(model, test_loader, device)

if __name__ == "__main__":
    main()

/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet50_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet50_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/resnet50-0676ba61.pth" to /root/.cache/torch/hub/checkpoints/resnet50-0676ba61.pth
100%|██████████| 97.8M/97.8M [00:00<00:00, 202MB/s]


Epoch [1/10], Step [20/129], Loss: 1.9599
Epoch [1/10], Step [40/129], Loss: 2.2884
Epoch [1/10], Step [60/129], Loss: 1.6255
Epoch [1/10], Step [80/129], Loss: 1.7584
Epoch [1/10], Step [100/129], Loss: 1.6858
Epoch [1/10], Step [120/129], Loss: 1.2583
Epoch [2/10], Step [20/129], Loss: 1.3293
Epoch [2/10], Step [40/129], Loss: 1.3817
Epoch [2/10], Step [60/129], Loss: 1.1013
Epoch [2/10], Step [80/129], Loss: 1.4492
Epoch [2/10], Step [100/129], Loss: 1.2861
Epoch [2/10], Step [120/129], Loss: 0.8878
Epoch [3/10], Step [20/129], Loss: 0.8861
Epoch [3/10], Step [40/129], Loss: 0.9240
Epoch [3/10], Step [60/129], Loss: 0.9457
Epoch [3/10], Step [80/129], Loss: 0.8526
Epoch [3/10], Step [100/129], Loss: 1.0237
Epoch [3/10], Step [120/129], Loss: 0.7340
Epoch [4/10], Step [20/129], Loss: 0.5977
Epoch [4/10], Step [40/129], Loss: 0.4394
Epoch [4/10], Step [60/129], Loss: 0.5436
Epoch [4/10], Step [80/129], Loss: 0.7212
Epoch [4/10], Step [100/129], Loss: 0.7402
Epoch [4/10], Step [120/129